In [ ]:
import random
import numpy as np
import pandas as pd
class state():
    def __init__(self, value, message, num):
        self.state_value = value
        self.state_message = message
        # 다음 Action 확률 정책
        self.next_states_IF_UP = [1 for i in range(num)]
        self.next_states_IF_DOWN = [1 for i in range(num)]
        # 연결리스트


class policy():
    def __init__(self, num):
        self.num = num
        self.policy_for_states = []
        self.set_policy_for_states(num)
        self.previous_path = []
        self.previous_value = set([])
        self.next_states_IF_FIRST = [1 for i in range(num)]
    
    def set_policy_for_states(self, num):
        for i in range(1, num + 1):
            self.policy_for_states.append(state(value=i, message='', num=num))

    def attack(self, target):
        # 다음 숫자 선택
        seed = random.random()
        sum_per = 0
        if not self.previous_path:
            policy_for_message = self.next_states_IF_FIRST
        elif self.previous_path[-1].state_message == 'UP':
            policy_for_message = self.previous_path[-1].next_states_IF_UP
        elif self.previous_path[-1].state_message == 'DOWN':
            policy_for_message = self.previous_path[-1].next_states_IF_UP
        for i in range(self.num):
            sum_per += policy_for_message[i] / sum(policy_for_message)
            if seed < sum_per and i + 1 not in self.previous_value:
                self.previous_path.append(self.policy_for_states[i])
                self.previous_value.add(i + 1)
                break
        # 업다운 메시지 받기
        if target == self.previous_path[-1].state_value:
            self.previous_path[-1].state_message = 'WIN'
        elif target > self.previous_path[-1].state_value:
            self.previous_path[-1].state_message = 'UP'
        else:
            self.previous_path[-1].state_message = 'DOWN'
        # print(f'정답 {target} : {self.previous_path[-1].state_value} {self.previous_path[-1].state_message}')
        # 역전파
        if self.previous_path[-1].state_message == 'WIN':
            next_state_value = self.previous_path[-1].state_value
            discount_rate = 0.1
            for st in reversed(self.previous_path):
                state_message = st.state_message
                if state_message == 'WIN':
                    continue
                if state_message == 'UP':
                    st.next_states_IF_UP[next_state_value - 1] += st.next_states_IF_UP[next_state_value - 1] * discount_rate
                elif state_message == 'DOWN':
                    st.next_states_IF_DOWN[next_state_value - 1] += st.next_states_IF_DOWN[next_state_value - 1] * discount_rate
                next_state_value = st.state_value - 1
                discount_rate *= discount_rate
            print(f'시도 횟수 : {len(self.previous_path)} / 기록 : {[i.state_value for i in self.previous_path]}')
            self.previous_path = []
            self.previous_value = set([])
        
        



def game(agent, target):
    while True:
        agent.attack(target)
        if not agent.previous_path:
            break



def main():
    print('숫자 범위를 입력하세요 : ')
    num = int(input())
    agent = policy(num)
    for i in range(1000000):
        game(agent, random.randrange(1, num + 1))

    # 확인
    # for st in agent.policy_for_states:
    #     print(st.state_value)
    #     print(st.state_message)
    #     print(st.next_states_IF_FIRST)
    #     print(st.next_states_LINKED_LIST)
    #     print('\n')
    # for i in range(5000000):
    #     target = random.randint(1, num + 1)
    #     game(agent, target)

if __name__ == "__main__":
    main()